## Imports

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mog_model import *
from utils.distributions import *
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice
from lattice.fields import SU2, SU3
from utils.dynamics import *
from utils.lattice_dynamics import *
from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, binarize, normal_kl, acl_spectrum, ESS
from utils.network import network, Linear, Sequential, Zip, Parallel, ScaleTanh
from utils.sampler import propose
from utils.notebook_utils import get_hmc_samples
from utils.data_utils import calc_avg_vals_errors, block_resampling, jackknife_err
from matplotlib import gridspec
from utils.plot_helper import errorbar_plot
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')

plt.rc('text', usetex=False)
plt.rcParams['errorbar.capsize'] = 0
%matplotlib notebook
%autoreload 2

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
def restart_sess(sess):
    tf.reset_default_graph()
    sess.close()
    return tf.InteractiveSession()

## Create IsingLattice

In [ ]:
sess = restart_sess(sess)

In [ ]:
lattice = IsingLattice(3, 4)

In [ ]:
samples = [lattice._randomize() for _ in range(10)]

In [ ]:
batch_size = len(samples)
energy_fn = lattice.get_energy_function()
dynamics = Dynamics(x_dim=lattice.num_sites, energy_function=energy_fn, 
                    trajectory_length=2, eps=0.1, hmc=False, 
                    net_factory=network, eps_trainable=True,
                    use_temperature=True)

In [ ]:
energies = lattice.calc_energy(samples, batch_size=len(samples))

In [ ]:
energies

In [ ]:
x_shape = tuple([len(samples)] + list(lattice.sites.shape))
x = tf.placeholder(tf.float32, shape=x_shape, name='x')
z = tf.random_normal(tf.shape(x), name='z')

In [ ]:
energy_fn = dynamics.energy(x, batch_size)

In [ ]:
print(sess.run(energy_fn, feed_dict={x: samples, dynamics.temperature: 1.}))

In [ ]:
kinetic = dynamics.kinetic(z)
_kinetic = sess.run(kinetic)

In [ ]:
lattice.sites.shape

In [ ]:
_kinetic.shape

In [ ]:
z2 = sess.run(tf.square(z))

In [ ]:
print(sess.run(tf.reduce_sum(tf.square(z), axis=z.shape)))

In [ ]:
tf.reduce_sum(tf.square(z), axis=)

In [ ]:
kinetic_ = tf.reduce_sum(tf.square(z), axis

In [ ]:
H = dynamics.hamiltonian(x, z, batch_size)

In [ ]:
print(sess.run(H, feed_dict={x: samples, dynamics.temperature: 1.}))

In [ ]:
def eager_execution():
    dataset = tf.data.Dataset.from_tensor_slices(x)
    iterator = dataset.make_initializable_iterator()

    next_element = iterator.get_next()

    #dynamics_energy = dynamics.energy(, aux=None)
    sess.run(iterator.initializer, feed_dict={x: samples})
    for i in range(len(samples)):
        value = sess.run(next_element)
        energy = sess.run(dynamics.energy(value), 
                          feed_dict={dynamics.temperature: 1.})
        print(energy)
        #print(value.shape)
        #_energy = sess.run(dynamics.energy(, aux=None))
        #print(_energy)

    sess.run(iterator.initializer, feed_dict={x: samples})
    for i in range(len(samples)):
        next_elt = sess.run(next_element)
        _step = sess.run(dynamics._forward_step(next_elt, z, i, aux=None),
                         feed_dict={dynamics.temperature: 1.})
        print(_step)

In [ ]:

hamiltonian = dynamics.hamiltonian(x, z, aux=None)

In [ ]:
hamiltonian_out = sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature: 1.})

In [ ]:
hamiltonian_out[0]

In [ ]:
step1 = dynamics._forward_step(x, z, 1, aux=None)

In [ ]:
Lx, _, px, output = propose(x, dynamics, do_mh_step=True)
Lz, _, pz, _ = propose(z, ldynamics, do_mh_step=True)

loss = tf.Variable(0., trainable=False, name='loss')
v1 = ((tf.reduce_sum(tf.square(x - Lx), axis=1) * px) + 1e-4)
v2 = ((tf.reduce_sum(tf.square(z - Lz), axis=1) * pz) + 1e-4)
scale = 0.1
loss = loss + scale * (tf.reduce_mean(1.0 / v1) + tf.reduce_mean(1.0 / v2))
loss = loss + ((-tf.reduce_mean(v1, name='v1')
                - tf.reduce_mean(v2, name='v2')) / scale)

In [ ]:
%debug

In [ ]:
step1 = ldynamics._forward_step(x, z, 1, aux=None)

In [ ]:
with tf.Session() as sess:
    print(sess.run(lattice.total_action(lattice.links)))

In [ ]:
energy = ldynamics.energy(x)
with tf.Session() as sess:
    print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature:1.}))

## LatticeDynamics testing

### Init tf.Session()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.close()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess.run(tf.global_variables_initializer())

### Create LatticeDynamics object

In [ ]:
_x_shape = tuple([None] + list(lattice.links.shape) + list(lattice.link_shape))
_x = tf.placeholder(dtype=tf.complex64, shape=_x_shape)

_links_samples = []
for _ in range(10):
    _links_sample = np.zeros(_x_shape[1:], dtype=np.complex64)
    for link in lattice.iter_links():
        _links_sample[link] = lattice.link_type.get_random_element()
    _links_samples.append(_links_sample)

In [ ]:
lattice.num_links

In [ ]:
_mask = sess.run(ldynamics.mask)

In [ ]:
_mask.reshape(tuple([-1] + list(lattice.links.shape)))

In [ ]:
m, _m = ldynamics._get_mask(0)

In [ ]:
np.array(links_samples).astype(np.complex64)

In [ ]:
links_samples = np.array(np.array(links_samples), dtype=np.complex64)

In [ ]:
kinetic = ldynamics.kinetic(z)
print(sess.run(kinetic), feed_dict={x: np.array(links_samples), dynamics.temperature: 1.})

In [ ]:
hamiltonian = ldynamics.hamiltonian(x, z)
sess.run(hamiltonian, feed_dict{x:})

In [ ]:
temp = np.array([1.0], dtype=np.float32)[0]
energy = ldynamics.energy(x)
print(sess.run(energy, feed_dict={x: links_samples, ldynamics.temperature: temp}))

In [ ]:
print(sess.run(energy, feed_dict={x: links_samples, ldynamics.temperature: temp}))

In [ ]:
print(sess.run(ldynamics.lattice.total_action()))
#ldynamics.lattice.total_action()

In [ ]:
forward_step = ldynamics._forward_step(x, z, 1, aux=None)

In [ ]:
%debug

In [ ]:
step1 = sess.run(forward_step, feed_dict={x: samples, ldynamics.temperature: 2.})

In [ ]:
x1, v1, log_jac_contrib1 = step1

In [ ]:
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: samples, dynamics.temperature:2.}))
print(sess.run(energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
print(sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(hamiltonian, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
grad_energy = dynamics.grad_energy(x)
print(sess.run(grad_energy, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(grad_energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
X, V, p_accept = dynamics.forward(x)

In [ ]:
_X = sess.run(X, feed_dict={x: samples, dynamics.temperature:2.})
print(_X.shape)

In [ ]:
_V = sess.run(V, feed_dict={x: samples, dynamics.temperature:2.})
print(_V.shape)

In [ ]:
_p_accept = sess.run(p_accept, feed_dict={x: samples, dynamics.temperature:2.})
print(_p_accept.shape)
print(_p_accept)